In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder

In [12]:
import os

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# Hyper-parameters 
num_epochs = 3
batch_size = 128
learning_rate = 0.001

In [4]:
#train and test data directory
train_dir = r"D:\Python Projects\AgePredictor\train"
test_dir = r"D:\Python Projects\AgePredictor\test"

In [28]:
# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Compose([transforms.Resize((32,32))])])

In [29]:
#load the train and test data
dataset = ImageFolder(train_dir,transform=transform)
test_dataset = ImageFolder(test_dir,transform=transform)

In [30]:
#initiate data loaders
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

In [31]:
classes = os.listdir("test")
classes

['0to10', '11to20', '21to30', '31to40', '41to60', '61plus']

In [32]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #3 color channels, output is 6, kernel is 5
        self.pool = nn.MaxPool2d(2, 2) #stride is 2
        self.conv2 = nn.Conv2d(6, 16, 5) #input 6 must be equal to last output size
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # afer both convulation and pooling layers the size is 16*5*5
        self.fc2 = nn.Linear(120, 84) # now it has been flatnned and is like a regular NN
        self.fc3 = nn.Linear(84, 6)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400 tensor is flattened
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 6
        return x

In [33]:
model = ConvNet().to(device)

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [35]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print('Finished Training')


Finished Training


In [40]:
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            #print(batch_size)
            try:
                label = labels[i]
                pred = predicted[i]
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1
            except  IndexError:
                pass

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    

Accuracy of the network: 32.813159004639395 %


In [42]:
for i in range(6):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of 0to10: 0.0 %
Accuracy of 11to20: 0.0 %
Accuracy of 21to30: 100.0 %
Accuracy of 31to40: 0.0 %
Accuracy of 41to60: 0.0 %
Accuracy of 61plus: 0.0 %


In [48]:
dummy_input = torch.randn(1, 3, 32, 32).to(device)
input_names = [ "actual_input" ]
output_names = [ "output" ]
model.eval()

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=6, bias=True)
)

In [49]:
# export as onnx
torch.onnx.export(model, 
                  dummy_input,
                  "agepred.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names,
                  export_params=True,
                  )